# X, Error 
### [[4, 9.266569064342626], 
### [8, 4.691634722238834], 
### [16, 2.3540305868408007], 
### [32, 1.1777906331299344],
### [64, 0.5889869374462385], 
### [128, 0.29447194065679233], 
### [256, 0.14728099599790476], 
### [512, 0.07368844371073896],
### [1024, 0.036855488027733516],
### [2048, 0.021444606921672753],
### [4096, 0.012765058924897349]]

<html>
    <h1>Libraries used :</h1>
    <ul style="list-style-type: none;">
        <li>
             <img width=150px height=150px  src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/54/Sympy_logo.svg/1200px-Sympy_logo.svg.png"  alt="Sympy">
        </li>
        <br>
        <li>
             <img width=300px height=300px  src="https://matplotlib.org/_static/logo2.png" alt="Matplotlib">
        </li>
        <br>
        <li>
             <img width=200px height=200px src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/NumPy_logo.svg/1200px-NumPy_logo.svg.png" alt="Numpy">
        </li>
        <br>
        <li>        
            <img width=200px height=200px  src="https://www.fullstackpython.com/img/logos/scipy.png" class="rounded" alt="Scipy">
        </li>
        <br>
        <li>
             <img width=400px height=400px  src="https://pandas.pydata.org/_static/pandas_logo.png" class="rounded" alt="Pandas">
        </li>
      </ul>

In [63]:
import numpy as np
from numba import jit, int32, float32, prange
from scipy.sparse import diags
from scipy.integrate import RK45, solve_ivp
import scipy.sparse as sp
import matplotlib.pyplot as plt
import matplotlib
from time import time
import pandas as pd

import time
import os
import math
#import pandas as pd
from tqdm import tqdm_notebook


In [64]:
plt.ioff()
plt.clf()

<html>
    <h3 align="center" style="color:#FF1C0C;"><u> Heat Equation </u> </h3> 
   <br>
</html>

\begin{equation}
\\\ u_t  + \beta u_{xx}    = - \left(\pi^{2} \beta - 1\right) e^{t} \sin{\left(\pi x \right)}
\\ u^h = \sum_{i= 0}^{n} a_i(t)  \phi_i(x) 
\\ \int_0^1 { (w u_t + \beta w u_{xx})} \ dx= \int_0^1 w ({- \left(\pi^{2} \beta - 1\right) e^{t} \sin{\left(\pi x \right)}}) dx
\\\ \int_0^1  w u_t   \ dx   - \beta \int_0^1 {w_x u_x} \ dx  + \beta \ (w(1) u_x(1) - w(0) u_x(0))= \int_0^1 w ({- \left(\pi^{2} \beta - 1\right) e^{t} \sin{\left(\pi x \right)}}) dx
\\  A \alpha^{'}(t) \hspace{5mm} \ -  \hspace{5mm}\beta  D \alpha(t) \hspace{10mm} +\hspace{10mm}  \beta E \alpha(t)   = f(t)
\\ w = {\phi_j}(x)
\end{equation}

\begin{equation}
\\  \alpha^{'}(t) = A^{-1}(\beta ( D - E) \alpha(t) + f(t)) \hspace{10mm} 
\end{equation}

<html>
    <h3 align="center" style="color:#FF1C0C;"><u>Boundary Conditions</u> </h3> <br>
</html>
\begin{equation}
\\  u(0, t) = 0  
\\  u(N, t) = 0
\\  u(x, 0) = sin(\pi x) 
\end{equation}

<html>
    <h3 align="center" style="color:#FF1C0C;"><u>Exact Solution</u> </h3> <br>
</html>
\begin{equation}
\\  u(x, t) = e^t sin (\pi x) 
\end{equation}

In [77]:

class Heat:
    
    #A *alpha - beta(D *) +beta(E(1 - 0)alpha) = 0
    

    def __init__(self, N=10):
        
        self.N = N
        self.H = 1/N
        self.domain = np.linspace(0, 1, N+1) 
        self.beta = -1
        self.t = 1
        self.b_c_start = 0
        self.b_c_end = 0
        
        def c_dash_transform(l):
            
            l[0] /= self.A[0]
            for i in range(1, self.N):
                  l[i] = (l[i])/ (self.A[i]-(self.H/6)* l[i-1])
                    
            return l        
  

                
        self.A = self.H* np.array([1/3] + [2/3]*(N-1) + [1/3])
        #         self.A = (self.H) * sp.csc_matrix(diags([       [1/6 for i in range(N)],   \
        #                                         [1/3] + [2/3]*(N-1) + [1/3], \
        #                                              [1/6 for i in range(N)] ], \
        #                                                    [1, 0, -1]), dtype=np.float32)
        
        self.c_dash = c_dash_transform(self.H *np.array( [1/6]*(N)))
        
        #-beta*
        
        
        self.D = N * sp.csr_matrix(diags([      [-1 for i in range(N)],  \
                                      [1] + [2]*(N-1) + [1], \
                                        [-1 for i in range(N)]],  \
                             [1, 0, -1] ), dtype=np.float32)
        self.D[0, 0] -= N     
        self.D[0, 1] -= -N
        self.D[N, N-1] -= -N
        self.D[N, N] -= N
        
        def f(k):
          
            if k == 0:
                return (1 + math.pi**2)*(math.pi*self.H - math.sin(math.pi*self.H))/(math.pi**2*self.H)
            elif k == N:
                return -(1 + math.pi**2)*(-math.pi*self.H + math.sin(math.pi*self.H))/(math.pi**2*self.H)
            else:
                return -2*(1 + math.pi**2)*(math.cos(math.pi*self.H) - 1)*math.sin(math.pi*self.H*k)/(math.pi**2*self.H)
        
        self.F = np.array([f(k) for k in prange(N+1)]).reshape(-1, 1) #incomplete without exp(t)
        

       
        #Set initial conditions here
        self.alpha = np.sin(np.pi* self.domain)
    
        #Set solution here
        self.exact = math.exp(self.t) * self.alpha

        
    def error(self):
        return np.linalg.norm(self.exact -  self.approx, 2)

    
    def apply_bc(self,  y):
        y[0, 0] = self.b_c_start
        y[-1, 0] = self.b_c_end
        return y
        
    
    def func (self, t,  y):
        #       self.A * y = (self.beta*(self.D* y) + math.exp(t)*self.F) = d 
        d = self.beta*self.D*y + math.exp(t)*self.F
        #A *alpha' = d form 
        def simplifier( d):    
            d[0] /= self.A[0]
            for i in range(1, self.N + 1):
                d[i, 0] = (d[i, 0] - (self.H/6)*d[i-1])/(self.A[i]-(self.H/6)*self.c_dash[i-1])
            for i in range(self.N-1, -1, -1): # Last entry is left unchanged ;see that i+1
                d[i, 0] -= self.c_dash[i]*d[i+1]                
        
            return self.apply_bc(d)                   
        
        #y = self.A*(self.beta*(self.D* y) + math.exp(t)*self.F)
        return simplifier(d)


    
    def algo_1(self):

        self.algo = solve_ivp(self.func,t_span=(0, 1),  y0=self.alpha, \
                   t_eval= np.array([1]), vectorized=True, method="LSODA")
        self.approx = self.apply_bc(self.algo.y).reshape(-1, )
        

In [82]:
Ns = [2**r for r in range(2, 9)]
l =[]
for i in tqdm_notebook(Ns):
    heateqn = Heat(i)

    t0 = time.time()

    heateqn.algo_1()
    l.append([i, heateqn.error()])
    if i!= Ns[-1]:
        del heateqn
    print(f"Time taken for algo to run for {i} basis",  time.time()-t0)
    


Time taken for algo to run for 4 basis 0.030004501342773438
Time taken for algo to run for 8 basis 0.030999422073364258
Time taken for algo to run for 16 basis 0.05698752403259277
Time taken for algo to run for 32 basis 0.1464402675628662
Time taken for algo to run for 64 basis 0.40776729583740234
Time taken for algo to run for 128 basis 1.4497451782226562
Time taken for algo to run for 256 basis 5.675202369689941



In [83]:
%matplotlib widget
errrs = []
xs = []
for i, j in l:
    xs.append(i)
    errrs.append(j)

xs, errrs =  np.array(xs), np.array(errrs)
slope, intercept = np.polyfit(-np.log(xs), np.log(errrs), 1)

plt.loglog(1/np.array(xs), errrs, '--')
plt.xlabel("H")
plt.ylabel("Error")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Error')

In [84]:
print(slope)

1.2174455681258682


In [85]:
df = pd.DataFrame(data = {"N":xs , "H": 1/xs, "Errors":errrs})
df.to_csv("Lin Hom ")